# 优化算法

优化算法的功能，是通过最小化或最大化目标函数，对模型的训练和表达能力造成影响的参数进行计算和更新，使这些参数达到或尽可能接近最优值，从而改善模型的训练，提高模型的学习能力。

## 欠拟合和过拟合

深度学习中，主要从以下两个角度来评价学习算法效果的好坏：

- 降低训练集上的误差，即训练误差。

- 减少训练集上的误差和测试集上的误差的差距。

这两个角度体现了机器学习面临的两个主要挑战：欠拟合和过拟合。

欠拟合是指模型不能在训练集上获得足够低的误差，即模型在训练集上的误差比人类水平达到的误差要高，此时模型还有提升的空间，可以通过增加模型深度和训练次数或选择一些优化算法继续提高模型的表现能力。

而过拟合是指学习时选择的模型所包含的参数过多，以至于这一模型对已知数据预测得很好，但对未知数据预测得很差的现象。通常称为模型的泛化能力不好，可以通过增加数据集、加入一些正则化方法或者改变超参数来进行调整。

下面我们针对过拟合的情况，首先对Dropout与Batch normalization进行简单介绍，之后分别使用Dropout和Batch normalization对第六章中的CNN与数字识别案例进行优化。


## Dropout

Dropout是通过修改模型本身结构来实现的，计算方便但功能强大。如图所示的三层人工神经网络：

<img src="image/dropout1.png" style="width:250px;height:250px;">

对于上图所示的网络，在训练开始时，按照一定地概率随机选择一些隐藏层神经元进行删除，即认为这些神经元不存在，这样便得到如下图的网络： 

<img src="image/dropout2.png" style="width:250px;height:250px;">

按照这样的网络计算梯度，进行参数更新（对删除的神经元不更新）。在下一次迭代时，再随机选择一些神经元，重复上面的做法，直到训练结束。
Dropout也可以看作是一种集成（bagging）方法，每次迭代的模型都不一样，最后以某种权重平均起来，这样参数的更新不再依赖于某些共同作用的隐层节点之间的关系，能够有效地防止过拟合。

## Batch normalization

机器学习的一个假设就是，数据是满足独立同分布的。而在深度学习模型中，原本做好预处理的同分布数据在经过层层的前向传导后，分布不断发生变化。随着网络的加深，上述变化带来的影响不断被放大。

Batch normalization的目的就是对网络的每一层输入做一个处理，使得它们尽可能满足输入同分布的基本假设。

可以对每一层的输入做标准化处理，使得输入均值为0方差为1：

$$\hat{x}^{(k)}=\frac{x^k - E[x^{(k)}]}{\sqrt{Var[x^{(k)}]}}$$

但如果只是简单地对每一层做白化处理，会降低层的表达能力。比如下图，在使用sigmoid激活函数的时候，如果把数据限制到0均值单位方差，那么相当于只使用了激活函数中近似线性的部分，这显然会降低模型表达能力。
<img src="image/batch_normalization.png" style="width:300px;height:200px;">


## 1 - 引用库

首先，载入几个需要用到的库，它们分别是：
- numpy：一个python的基本库，用于科学计算
- matplotlib.pyplot：用于生成图，在验证模型准确率和展示成本变化趋势时会使用到
- PIL：用于最后使用自己的图片验证训练模型
- paddle.v2：PaddlePaddle深度学习框架
- paddle.v2.plot：PaddlePaddle深度学习框架的绘图工具

In [1]:
import matplotlib
import os
from PIL import Image
import numpy as np
import paddle.v2 as paddle
from paddle.v2.plot import Ploter

添加一些绘图相关标注，在绘制学习曲线时将用到。

In [2]:
with_gpu = os.getenv('WITH_GPU', '0') != '0'

step = 0

# 绘图相关标注
train_title_cost = "Train cost"
test_title_cost = "Test cost"

train_title_error = "Train error rate"
test_title_error = "Test error rate"

## 2 - 定义卷积神经网络分类器

** 普通的卷积神经网络分类器 **

首先，我们定义一个普通的卷积神经网络分类器convolutional_neural_network()，它的结构为卷积层-池化层-卷积层-池化层-全连接层组成，在PaddlePaddle中将一个卷积层和一个池化层当做一层，并使用paddle.networks.simple_img_conv_pool()来定义一个卷积-池化层，其中的各个参数分别表示：

- input：输入数据
- filter_size：卷积核大小
- num_filters：卷积核数量
- num_channel：卷积核通道数
- pool_size：池化层大小
- pool_stride：池化层步长
- act：激活函数，这里我们采用Relu()激活函数



In [3]:
def convolutional_neural_network(img):
    """
    定义卷积神经网络分类器：
        输入的二维图像，经过两个卷积-池化层，使用以softmax为激活函数的全连接层作为输出层
    Args:
        img -- 输入的原始图像数据
    Return:
        predict -- 分类的结果
    """
    # 第一个卷积-池化层
    conv_pool_1 = paddle.networks.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        num_channel=1,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())

    # 第二个卷积-池化层
    conv_pool_2 = paddle.networks.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        num_channel=20,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())
    # 全连接层
    predict = paddle.layer.fc(
        input=conv_pool_2, size=10, act=paddle.activation.Softmax())
    return predict


** 使用Dropout优化的卷积神经网络分类器 **

定义一个使用Dropout优化的卷积神经网络分类器convolutional_neural_network_with_dropout()，它的结构与普通的卷积神经网络分类器相同，但是在每个卷积-池化层中都加入了dropout设置，在PaddlePaddle中使用conv_layer_attr=paddle.attr.ExtraLayerAttribute(drop_rate=0.5)来在卷积-池化层中添加dropout并设置drop_rate=0.5。

In [4]:
def convolutional_neural_network_with_dropout(img):
    """
    定义卷积神经网络分类器：
        输入的二维图像，经过两个卷积-池化层，使用以softmax为激活函数的全连接层作为输出层
    Args:
        img -- 输入的原始图像数据
    Return:
        predict -- 分类的结果
    """
    """
    不同之处：
        在两个卷积-池化层中加入了dropout设置
    """
    # 第一个卷积-池化层
    conv_pool_1 = paddle.networks.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        num_channel=1,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu(),
        conv_layer_attr=paddle.attr.ExtraLayerAttribute(drop_rate=0.5))

    # 第二个卷积-池化层
    conv_pool_2 = paddle.networks.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        num_channel=20,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu(),
        conv_layer_attr=paddle.attr.ExtraLayerAttribute(drop_rate=0.5))
    # 全连接层
    predict = paddle.layer.fc(
        input=conv_pool_2, size=10, act=paddle.activation.Softmax())
    return predict


** 使用Batch normalization优化的卷积神经网络分类器 **

定义一个使用Batch normalization优化的卷积神经网络分类器convolutional_neural_network_with_batch_norm()，它的结构与普通的卷积神经网络分类器相同，但是在每个卷积-池化层之后加入Batch normalization操作，在PaddlePaddle中使用paddle.layer.batch_norm(input=conv_pool_1, act=paddle.activation.Relu()来添加Batch normalization。

In [5]:
def convolutional_neural_network_with_batch_norm(img):
    """
    定义卷积神经网络分类器：
        输入的二维图像，经过两个卷积-池化层，使用以softmax为激活函数的全连接层作为输出层
    Args:
        img -- 输入的原始图像数据
    Return:
        predict -- 分类的结果
    """
    """
    与第六章代码不同之处：
        在两个卷积-池化层之后都加入了batch normalization层norm1和norm2
    """
    # 第一个卷积-池化层
    conv_pool_1 = paddle.networks.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        num_channel=1,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())

    norm1 = paddle.layer.batch_norm(input=conv_pool_1, act=paddle.activation.Relu())
    
    # 第二个卷积-池化层
    conv_pool_2 = paddle.networks.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        num_channel=20,
        pool_size=2,
        pool_stride=2,
        act=paddle.activation.Relu())

    norm2 = paddle.layer.batch_norm(input=conv_pool_2, act=paddle.activation.Relu())
        
    # 全连接层
    predict = paddle.layer.fc(
        input=norm2, size=10, act=paddle.activation.Softmax())
    return predict

## 3 - 配置网络结构 

调用分类器（这里我们提供了三个不同的分类器，大家可以试着使用不同的分类器进行训练）得到分类结果。训练时，对该结果计算其损失函数，分类问题常常选择交叉熵损失函数。

指定训练相关的参数。
- 训练方法（optimizer)： 代表训练过程在更新权重时采用动量优化器 `Momentum` ，其中参数0.9代表动量优化每次保持前一次速度的0.9倍。
- 训练速度（learning_rate）： 迭代的速度，与网络的训练收敛速度有关系。
- 正则化（regularization）： 是防止网络过拟合的一种手段，此处采用L2正则化。

In [6]:
def netconfig():
    """
    配置网络结构
    Args:
    Return:
        images -- 输入层
        label -- 标签数据
        predict -- 输出层
        cost -- 损失函数
        parameters -- 模型参数
        optimizer -- 优化器
    """
    
    """
    输入层:
        paddle.layer.data表示数据层,
        name=’pixel’：名称为pixel,对应输入图片特征
        type=paddle.data_type.dense_vector(784)：数据类型为784维(输入图片的尺寸为28*28)稠密向量
    """
    images = paddle.layer.data(
        name='pixel', type=paddle.data_type.dense_vector(784))
        
    """
    数据层:
        paddle.layer.data表示数据层,
        name=’label’：名称为label,对应输入图片的类别标签
        type=paddle.data_type.dense_vector(10)：数据类型为10维(对应0-9这10个数字)稠密向量
    """
    label = paddle.layer.data(
        name='label', type=paddle.data_type.integer_value(10))
    
    # 使用普通的卷积神经网络
    predict = convolutional_neural_network(images)
    
    # 使用带dropout优化的卷积神经网络
#     predict = convolutional_neural_network_with_dropout(images)
    
    # 使用带batch_norm优化的卷积神经网络
#     predict = convolutional_neural_network_with_batch_norm(images)

    # 定义成本函数，addle.layer.classification_cost()函数内部采用的是交叉熵损失函数
    cost = paddle.layer.classification_cost(input=predict, label=label)

    # 利用cost创建参数parameters
    parameters = paddle.parameters.create(cost)
      
    # 创建优化器optimizer，下面列举了2种常用的优化器，不同类型优化器选一即可
    # 创建Momentum优化器，并设置学习率(learning_rate)、动量(momentum)和正则化项(regularization)
    """
    与第六章代码不同之处：
        学习率learning_rate和动量momentum设置的数值不同，
            一方面，可以通过单纯修改某个参数值而不引入其他改变，对比第六章实验结果来验证该参数的影响;
            另一方面，可以通过设置learning_rate=0.1 / 128.0，momentum=0.95，以使得模型的基础表现相对第六章中下降，如收敛程度或者速度下降
                而进一步加入新的模块或者设置后（如加入dropout），模型表现得到提升，从而验证新加入的模块或者设置的有效性;
    """
    optimizer = paddle.optimizer.Momentum(
        learning_rate=0.1 / 128.0,
        momentum=0.95,
        regularization=paddle.optimizer.L2Regularization(rate=0.0005 * 128))
    
    # 创建Adam优化器，并设置参数beta1、beta2、epsilon
    # optimizer = paddle.optimizer.Adam(beta1=0.9, beta2=0.99, epsilon=1e-06)
    
    config_data = [images, label, predict, cost, parameters, optimizer]
    
    return config_data


## 4 - 训练模型

下面，我们开始训练模型，我们定义需要用到的工具函数，分别为plot_init()、load_image()、infer()用来绘制学习曲线、载入图片和预测。

In [9]:
def plot_init():
    """
    绘图初始化函数：
        初始化绘图相关变量
    Args:
    Return:
        cost_ploter -- 用于绘制cost曲线的变量
        error_ploter -- 用于绘制error_rate曲线的变量
    """
    # 绘制cost曲线所做的初始化设置
    cost_ploter = Ploter(train_title_cost, test_title_cost)
    
    # 绘制error_rate曲线所做的初始化设置
    error_ploter = Ploter(train_title_error, test_title_error)
    
    ploter = [cost_ploter, error_ploter]
    
    return ploter

    
def load_image(file):
    """
    定义读取输入图片的函数：
        读取指定路径下的图片，将其处理成分类网络输入数据对应形式的数据，如数据维度等
    Args:
        file -- 输入图片的文件路径
    Return:
        im -- 分类网络输入数据对应形式的数据
    """
    im = Image.open(file).convert('L')
    im = im.resize((28, 28), Image.ANTIALIAS)
    im = np.array(im).astype(np.float32).flatten()
    im = im / 255.0
    return im


def infer(predict, parameters, file):
    """
    定义判断输入图片类别的函数：
        读取并处理指定路径下的图片，然后调用训练得到的模型进行类别预测
    Args:
        predict -- 输出层
        parameters -- 模型参数
        file -- 输入图片的文件路径
    Return:
    """
    # 读取并预处理要预测的图片
    test_data = []
    cur_dir = os.getcwd()
    test_data.append((load_image(cur_dir + '/image/infer_3.png'),))
    
    # 利用训练好的分类模型，对输入的图片类别进行预测
    probs = paddle.infer(
        output_layer=predict, parameters=parameters, input=test_data)
    lab = np.argsort(-probs)
    print "Label of image/infer_3.png is: %d" % lab[0][0]

开始训练

In [8]:
# 初始化，设置是否使用gpu，trainer数量
paddle.init(use_gpu=with_gpu, trainer_count=1)
    
# 定义神经网络结构
images, label, predict, cost, parameters, optimizer = netconfig()

# 构造trainer,配置三个参数cost、parameters、update_equation，它们分别表示成本函数、参数和更新公式
trainer = paddle.trainer.SGD(
    cost=cost, parameters=parameters, update_equation=optimizer)
    
# 初始化绘图变量
cost_ploter, error_ploter = plot_init()
    
# lists用于存储训练的中间结果，包括cost和error_rate信息，初始化为空
lists = []

def event_handler_plot(event):
    """
    定义event_handler_plot事件处理函数：
        事件处理器，可以根据训练过程的信息做相应操作：包括绘图和输出训练结果信息
    Args:
        event -- 事件对象，包含event.pass_id, event.batch_id, event.cost等信息
    Return:
    """
    global step
    if isinstance(event, paddle.event.EndIteration):
        # 每训练100次（即100个batch），添加一个绘图点
        if step % 100 == 0:
            cost_ploter.append(train_title_cost, step, event.cost)
            # 绘制cost图像，保存图像为‘train_test_cost.png’
            cost_ploter.plot('./train_test_cost')
            error_ploter.append(
                train_title_error, step, event.metrics['classification_error_evaluator'])
            # 绘制error_rate图像，保存图像为‘train_test_error_rate.png’
            error_ploter.plot('./train_test_error_rate')
        step += 1
        # 每训练100个batch，输出一次训练结果信息
        if event.batch_id % 100 == 0:
            print "Pass %d, Batch %d, Cost %f, %s" % (
                event.pass_id, event.batch_id, event.cost, event.metrics)
    if isinstance(event, paddle.event.EndPass):
        # 保存参数至文件
        with open('params_pass_%d.tar' % event.pass_id, 'w') as f:
            trainer.save_parameter_to_tar(f)
        # 利用测试数据进行测试
        result = trainer.test(reader=paddle.batch(
            paddle.dataset.mnist.test(), batch_size=128))
        print "Test with Pass %d, Cost %f, %s\n" % (
            event.pass_id, result.cost, result.metrics)
        # 添加测试数据的cost和error_rate绘图数据
        cost_ploter.append(test_title_cost, step, result.cost)
        error_ploter.append(
            test_title_error, step, result.metrics['classification_error_evaluator'])
        # 存储测试数据的cost和error_rate数据
        lists.append((
            event.pass_id, result.cost, result.metrics['classification_error_evaluator']))
                
trainer.train(
    reader=paddle.batch(
        paddle.reader.shuffle(paddle.dataset.mnist.train(), buf_size=8192),
        batch_size=128),
    event_handler=event_handler_plot,
    num_passes=10)

# 在多次迭代中，找到在测试数据上表现最好的一组参数，并输出相应信息
best = sorted(lists, key=lambda list: float(list[1]))[0]
print 'Best pass is %s, testing Avgcost is %s' % (best[0], best[1])
print 'The classification accuracy is %.2f%%' % (100 - float(best[2]) * 100)
    
# 预测输入图片的类型
infer(predict, parameters, '/image/infer_3.png')


[INFO 2017-12-27 04:31:42,058 layers.py:2707] output for __conv_pool_0___conv: c = 20, h = 24, w = 24, size = 11520
[INFO 2017-12-27 04:31:42,062 layers.py:2849] output for __conv_pool_0___pool: c = 20, h = 12, w = 12, size = 2880
[INFO 2017-12-27 04:31:42,068 layers.py:2707] output for __conv_pool_1___conv: c = 50, h = 8, w = 8, size = 3200
[INFO 2017-12-27 04:31:42,075 layers.py:2849] output for __conv_pool_1___pool: c = 50, h = 4, w = 4, size = 800


Pass 0, Batch 0, Cost 3.024604, {'classification_error_evaluator': 0.9140625}
Pass 0, Batch 100, Cost 2.308956, {'classification_error_evaluator': 0.9453125}
Pass 0, Batch 200, Cost 2.307811, {'classification_error_evaluator': 0.9140625}
Pass 0, Batch 300, Cost 2.302251, {'classification_error_evaluator': 0.9140625}
Pass 0, Batch 400, Cost 2.308545, {'classification_error_evaluator': 0.90625}
Test with Pass 0, Cost 2.302830, {'classification_error_evaluator': 0.8865000009536743}

Pass 1, Batch 0, Cost 2.304678, {'classification_error_evaluator': 0.8828125}
Pass 1, Batch 100, Cost 2.300503, {'classification_error_evaluator': 0.8828125}
Pass 1, Batch 200, Cost 2.305115, {'classification_error_evaluator': 0.921875}
Pass 1, Batch 300, Cost 2.296468, {'classification_error_evaluator': 0.890625}
Pass 1, Batch 400, Cost 2.307123, {'classification_error_evaluator': 0.9140625}
Test with Pass 1, Cost 2.302628, {'classification_error_evaluator': 0.8865000009536743}

Pass 2, Batch 0, Cost 2.306701

## 总结

大家可以尝试使用三种不同的卷积神经网络来训练模型，可以发现三者的训练结果分别如下：

** 普通的卷积神经网络 **

<img src="image/default.png" style="width:500px;height:50px">

** 使用dropout优化的卷积神经网络 **

<img src="image/dropout.png" style="width:500px;height:50px">

** 使用Batch normalization优化的卷积神经网络**

<img src="image/norm.png" style="width:500px;height:50px">

我们发现普通的卷积神经网络效果十分差，请不要惊讶，这是因为我们调整了Momentum和Learning_rate来是这个基础模型的学习效率下降，从而让大家能够清晰地发现Dropout和Batch normalization能够提升训练效果，减少过拟合的情况。同时，由于我们调整了Momentum和Learning_rate这两个参数，使得模型效果变差，其实反过来也间接表示我们可以通过调整这两个参数来让模型的效果变好。